In [0]:
%%sh
# Download the CMS Open Payments ZIP file
curl -L https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip -o /tmp/openpayments_2023.zip

# Unzip the file to a directory
unzip /tmp/openpayments_2023.zip -d /tmp/openpayments_2023

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  752M  100  752M    0     0  93.4M      0  0:00:08  0:00:08 --:--:-- 93.5M


Archive:  /tmp/openpayments_2023.zip


replace /tmp/openpayments_2023/OP_PGYR2023_README_P01302025.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


---------------------------------------------------------------------------
CalledProcessError                        Traceback (most recent call last)
File <command-73876336522368>:1
----> 1 get_ipython().run_cell_magic('sh', '', 'curl -L https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip -o /tmp/openpayments_2023.zip\n\nunzip /tmp/openpayments_2023.zip -d /tmp/openpayments_2023\n\npayment_data = spark.read.csv("file:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)\n\npayment_data.write \\\n    .format("delta") \\\n    .mode("overwrite") \\\n    .saveAsTable("General_2023_OpenPayments")\n')

File /databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2362, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2360 with self.builtin_trap:
   2361     args = (magic_arg_s, cell)
-> 2362     result = fn(*args, **kwargs)
   2363 return result

File /databricks/python/lib/python3.9/s

In [0]:
# Reading the CSV file into a Spark DataFrame
payment = spark.read.csv("file:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

# Saving it as a Delta table
payment.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("General_2023_OpenPayments")

In [0]:
# File location and type
file_location = "/FileStore/tables/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
recipient = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
from pyspark.sql.functions import col, desc, sum, count

payment = payment.withColumn("Total_Amount_of_Payment_USDollars", 
                                   col("Total_Amount_of_Payment_USDollars").cast("double"))

In [0]:
joined_df = payment.join(
    recipient,
    payment["Covered_Recipient_Profile_ID"] == recipient["Covered_Recipient_Profile_ID"],
    "left"
)

In [0]:
large_payments = payment.filter(col("Total_Amount_of_Payment_USDollars") > 1000) \
    .groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(count("*").alias("count")) \
    .orderBy(desc("count"))
large_payments.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164093|
|Consulting Fee                                                                                                                                    |105239|
|Travel and Lodging                                                                                                                                |24793 |
|Honoraria                                                      

The category "Compensation for services other than consulting" appears in 164,093 payments that exceed $1,000 in value while "Consulting Fee" appears in 105,239 such payments. Non-consulting professional services make up the biggest category of large payments to healthcare providers while consulting arrangements stand as the second most frequently observed substantial compensation method.

In [0]:
payment_count = payment.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(count("*").alias("count")) \
    .orderBy(desc("count")) \
    .limit(10)
payment_count.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

Payments categorized under "Food and Beverage" exceed 13.3 million instances to become the leading payment type among all others. The payment category "Travel and Lodging" appears 545,086 times in the Open Payments database while "Compensation for services other than consulting" appears 236,628 times. The overwhelming number of small food and beverage payments demonstrates these transactions involve minimal value whereas the less frequent service payments represent higher worth.

In [0]:
payment_amount = payment.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(desc("total_amount")) \
    .limit(10)
payment_amount.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |total_amount        |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Royalty or License                                                                                                                                |1.19217456302E9     |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500002E8 |
|Consulting Fee                                                                                                                                    |5.

Payments under the "Royalty or License" category represent the highest total financial value at $1.19 billion although they are less common than other payment types. The payment categories "Compensation for services other than consulting" and "Consulting Fee" amount to approximately $595 million and $515 million respectively. Knowledge-based services together with intellectual property represent the highest financial value in the healthcare payment system.

In [0]:
specialty_amount = joined_df.filter(col("Covered_Recipient_Type") == "Covered Recipient Physician") \
    .groupBy("Covered_Recipient_Profile_Primary_Specialty") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(desc("total_amount")) \
    .limit(10)
specialty_amount.show(truncate=False)

+------------------------------------------------------------------------------------------------+--------------------+
|Covered_Recipient_Profile_Primary_Specialty                                                     |total_amount        |
+------------------------------------------------------------------------------------------------+--------------------+
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                         |3.777151620799988E8 |
|Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology                            |8.667850009999989E7 |
|Allopathic & Osteopathic Physicians|Neurological Surgery                                        |8.586441204999998E7 |
|Allopathic & Osteopathic Physicians|Dermatology                                                 |7.997300741000049E7 |
|null                                                                                            |7.51350700000005E7  |
|Allopathic & Osteopathic Physicians|Ort

Orthopedic Surgery specialists obtained the largest total payments reaching $378 million while Neurology specialists received $87 million as the second highest amount. The total payments for Neurological Surgery, Dermatology and Adult Reconstructive Orthopedic Surgery (specialized) combined to reach $73-86 million. The medical field shows that surgical specialties which perform implantable device procedures and complex surgeries receive significantly higher payments from industry sources than other medical specialties.

In [0]:
physician_amount = payment.filter(col("Covered_Recipient_Type") == "Covered Recipient Physician") \
    .groupBy(
        "Covered_Recipient_First_Name", 
        "Covered_Recipient_Middle_Name",
        "Covered_Recipient_Last_Name", 
        "Covered_Recipient_Profile_ID"
    ) \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(desc("total_amount")) \
    .limit(10)
physician_amount.show(truncate=False)

+----------------------------+-----------------------------+---------------------------+----------------------------+-----------------+
|Covered_Recipient_First_Name|Covered_Recipient_Middle_Name|Covered_Recipient_Last_Name|Covered_Recipient_Profile_ID|total_amount     |
+----------------------------+-----------------------------+---------------------------+----------------------------+-----------------+
|STEPHEN                     |null                         |BURKHART                   |288926                      |3.392202493E7    |
|WILLIAM                     |null                         |BINDER                     |1166415                     |2.943408911E7    |
|KEVIN                       |null                         |FOLEY                      |311622                      |1.730510451E7    |
|IVAN                        |null                         |OSORIO                     |105583                      |1.606551551E7    |
|GEORGE                      |null              

The physician who received the largest amount was Stephen Burkhart with $34 million while William Binder obtained $29 million and Kevin Foley received $17 million. A total of $127 million went to the top ten physicians who received payments yet the payments dropped dramatically after the fifth physician. The extremely concentrated industry payments to few doctors indicate these physicians possess valuable patents or serve as opinion leaders and have developed proprietary techniques in their fields.